In [ ]:
#!pip install triton==2.0.0.dev20221103

In [ ]:
#!pip3 install transformers==4.29

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from Bio import SeqIO
import math
import pandas as pd
from tqdm import tqdm
import os
import time
import numpy as np

In [2]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/home/rxa615/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rxa615/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing Be

In [3]:
import os
os.getcwd()

'/home/rxa615'

In [4]:
os.chdir('/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/datasets')

In [11]:
datasets_folder = '/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/datasets'
csv_files = [file for file in os.listdir(datasets_folder) if file.endswith('.csv')]

In [12]:
csv_files

['test_demo_coding_vs_intergenomic_seqs.csv',
 'test_human_enhancers_cohn.csv',
 'test_human_enhancers_ensembl.csv',
 'test_human_ensembl_regulatory.csv',
 'test_human_nontata_promoters.csv',
 'test_human_ocr_ensembl.csv',
 'train_demo_coding_vs_intergenomic_seqs.csv',
 'train_human_enhancers_cohn.csv',
 'train_human_enhancers_ensembl.csv',
 'train_human_ensembl_regulatory.csv',
 'train_human_nontata_promoters.csv',
 'train_human_ocr_ensembl.csv']

In [13]:
csv_files = csv_files[8:12]
print(csv_files)

['train_human_enhancers_ensembl.csv', 'train_human_ensembl_regulatory.csv', 'train_human_nontata_promoters.csv', 'train_human_ocr_ensembl.csv']


In [14]:
from tqdm import tqdm

# Define a function to calculate mean embedding
def calculate_mean_embedding(sequence):
    inputs = tokenizer(sequence, return_tensors='pt')["input_ids"]
    with torch.no_grad():
        hidden_states = model(inputs)[0]  # [1, sequence_length, 768]

    # Embedding with mean pooling
    embedding_mean = torch.mean(hidden_states, dim=1).squeeze()  # Calculate mean along the sequence length
    # Convert torch tensor to numpy array
    embedding_mean_np = embedding_mean.detach().numpy()

    return embedding_mean_np

In [ ]:
# Folder path to save the embeddings and the time calculation file
output_folder_path = "/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/DNA_BERT2"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# List to store the time taken for each file
time_records = []

# Apply the calculate_mean_embedding function to each CSV file
for csv_file in csv_files:
    file_path = os.path.join(datasets_folder, csv_file)
    print(csv_file)
    df = pd.read_csv(file_path)
    print(df.shape)
    # Measure the time taken to calculate the mean embeddings
    start_time = time.time()
    tqdm.pandas()
    df['mean_embedding'] = df['seq'].progress_apply(calculate_mean_embedding)
    end_time = time.time()
    time_taken = end_time - start_time

    # Save the time taken for the current file
    time_records.append({'file_name': csv_file, 'time_taken': time_taken})

    # Save the 'mean_embedding' column as a numpy array
    embedding_file_path = os.path.join(output_folder_path, os.path.splitext(csv_file)[0] + "_DB2_Embeddings.npy")
    np.save(embedding_file_path, df['mean_embedding'].values)

    print(f"Embeddings saved successfully at: {embedding_file_path}")

# # Save the time calculation records to a CSV file
# time_df = pd.DataFrame(time_records)
# time_csv_path = os.path.join(output_folder_path, "DB2_time_calculation.csv")
# time_df.to_csv(time_csv_path, index=False)

# print(f"Time calculations saved successfully at: {time_csv_path}")


train_human_enhancers_ensembl.csv
(123872, 2)


  1%|▏         | 1847/123872 [01:22<1:42:02, 19.93it/s]/home/rxa615/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:433: UserWarning: Increasing alibi size from 512 to 520
  warnings.warn(
 48%|████▊     | 59186/123872 [44:10<43:53, 24.56it/s]   /home/rxa615/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:433: UserWarning: Increasing alibi size from 520 to 549
  warnings.warn(
100%|██████████| 123872/123872 [1:32:07<00:00, 22.41it/s]


Embeddings saved successfully at: /.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/DNA_BERT2/train_human_enhancers_ensembl_DB2_Embeddings.npy
train_human_ensembl_regulatory.csv
(231348, 2)


 14%|█▍        | 32463/231348 [43:00<3:56:03, 14.04it/s]